In [6]:
import numpy as np
import math
import time

def read_tsp_file(file_path):
    city_coordinates = {}
    with open(file_path, 'r') as file:
        # Temukan awal dari NODE_COORD_SECTION
        for line in file:
            if line.strip() == "NODE_COORD_SECTION":
                break
        
        # Baca data koordinat kota
        for line in file:
            if line.strip() == "EOF":
                break
            # Split line and convert values to integers
            values = list(map(int, line.split()))
            
            if len(values) == 2:
                # If there are two values, assume (x, y) and use the line number as city_id
                city_id = len(city_coordinates) + 1
                x, y = values
            elif len(values) == 3:
                # If there are three values, assume (city_id, x, y)
                city_id, x, y = values
            else:
                # Handle other cases as needed
                continue
            
            city_coordinates[city_id] = (x, y)
    
    return city_coordinates

file_path = 'D:/att48.tsp'
city_coordinates = read_tsp_file(file_path)


# Function to calculate Euclidean distance between two cities
def euclidean_distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

def calculate_total_distance(path, city_coordinates):
    total_distance = 0
    for i in range(len(path) - 1):
        from_city = city_coordinates[path[i]]
        to_city = city_coordinates[path[i + 1]]
        total_distance += euclidean_distance(from_city, to_city)
    # Adding the distance from the last city back to the first city
    from_city = city_coordinates[path[-1]]
    to_city = city_coordinates[path[0]]
    total_distance += euclidean_distance(from_city, to_city)
    return total_distance


# Function to find the nearest neighbors of a given city
def get_nearest_neighbors(city_index, city_coordinates, neighbor_number):
    distances = []
    for index, coords in city_coordinates.items():
        if index != city_index:
            distance = euclidean_distance(city_coordinates[city_index], coords)
            distances.append((index, distance))
    distances.sort(key=lambda x: x[1])
    return [index for index, _ in distances[:neighbor_number]]

# Function to generate a greedy permutation of remaining cities
def greedy_permutation(remaining_cities, city_coordinates):
    if not remaining_cities:
        return []
    current_city = remaining_cities[0]
    path = [current_city]
    while len(path) < len(remaining_cities):
        next_city = min(
            remaining_cities, 
            key=lambda x: euclidean_distance(city_coordinates[path[-1]], city_coordinates[x]) if x not in path else float('inf')
        )
        path.append(next_city)
    return path

# Function to generate initial population using the GPM method
def generate_population_gpm(city_coordinates, population_size, neighbor_number):
    population = []
    for city_index in city_coordinates.keys():
        individual = [city_index]
        neighbors = get_nearest_neighbors(city_index, city_coordinates, neighbor_number)
        for neighbor in neighbors:
            individual.append(neighbor)
        remaining_cities = list(city_coordinates.keys())
        for city in individual:
            remaining_cities.remove(city)
        rest_cities = greedy_permutation(remaining_cities, city_coordinates)
        individual.extend(rest_cities)
        population.append(individual)
        if len(population) >= population_size:
            break
    return population


population_size = 100
neighbor_number = 3
# Generate population using GPM
population_gpm = generate_population_gpm(city_coordinates, population_size, neighbor_number)

# Print each individual in the population with a new line for each individual
for i, individual in enumerate(population_gpm):
    print(f"Individu {i + 1}: {individual}\n")

Individu 1: [1, 9, 8, 22, 2, 29, 5, 48, 39, 32, 21, 47, 11, 23, 14, 25, 13, 12, 15, 33, 46, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 20, 40, 3, 34, 41, 16, 38, 31, 42, 10, 24, 45, 35, 26, 4]

Individu 2: [2, 29, 42, 5, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 23, 14, 25, 13, 21, 47, 20, 40, 3, 22, 16, 41, 34, 48, 39, 32, 24, 10, 35, 45, 26, 4, 8]

Individu 3: [3, 22, 23, 34, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 14, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4, 41, 16, 8, 40, 20]

Individu 4: [4, 26, 10, 35, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 23, 14, 25, 13, 21, 47, 20, 40, 3, 22, 16, 41, 34, 29, 5, 48, 39, 32, 24, 42, 2, 45, 8]

Individu 5: [5, 48, 29, 42, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 23, 14, 25, 13, 21, 47, 20, 40, 3, 22, 16, 41, 34, 39, 32, 24, 10, 35, 45, 26, 4, 2, 8]

Individu 6: [6,

In [7]:
# Fitness evaluation
def evaluate_fitness(population, city_coordinates):
    fitness_scores = []
    for individual in population:
        distance = calculate_total_distance(individual, city_coordinates)
        fitness_score = 1 / distance  # Higher fitness for shorter path
        fitness_scores.append(fitness_score)
    return np.array(fitness_scores)

# Selection - Roulette Wheel Selection
def select_parents(population, fitness_scores, num_parents):
    parents = []
    for _ in range(num_parents):
        parent_idx = np.random.choice(np.arange(len(population)), p=fitness_scores/fitness_scores.sum())
        parents.append(population[parent_idx])
    return parents

def two_point_crossover(parent1, parent2):
    size = len(parent1)
    # Hasilkan dua titik berbeda untuk saling bersilangan
    crossover_points = sorted(np.random.choice(range(1, size), 2, replace=False))
    start, end = crossover_points
    
    # Initialize the child with None
    child = [None] * size
    
    # Salin segmen dari parent1 ke anak
    child[start:end] = parent1[start:end]
    # Isi sisa rute dari induk2, pastikan tidak ada duplikat
    current_position = end
    for city in parent2:
        if city not in child:
            if current_position >= size:
                current_position = 0
            child[current_position] = city
            current_position += 1  
    return child

# Mutation - Swap Mutation
def swap_mutation(individual, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(len(individual), 2, replace=False)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

# Genetic Algorithm for TSP
def genetic_algorithm_tsp_with_gpm(city_coordinates, population_size=100, num_generations=100, mutation_rate=0.01, elitism_size=2):
    start_time = time.time()
    population = generate_population_gpm(city_coordinates, population_size, 5)
    
    for generation in range(num_generations):
        #Evaluasi Fitness
        fitness_scores = evaluate_fitness(population, city_coordinates)
        total_fitness = np.sum(fitness_scores)
        best_fitness = np.max(fitness_scores)
        best_distance = 1 / np.max(fitness_scores)
        
        # Elitism
        elite_indices = np.argsort(-fitness_scores)[:elitism_size]
        elites = [population[i] for i in elite_indices]
        
        # Selection
        parents = select_parents(population, fitness_scores, population_size - elitism_size)
        
        # Crossover and mutation
        next_population = []
        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                child1 = two_point_crossover(parents[i], parents[i+1])
                child2 = two_point_crossover(parents[i+1], parents[i])
                next_population.extend([swap_mutation(child1, mutation_rate), swap_mutation(child2, mutation_rate)])
        
        # Elitism - Adding the elites back to the population
        next_population.extend(elites)
        
        # Update population
        population = next_population
        
        # Evaluate new population's fitness
        fitness_scores = evaluate_fitness(population, city_coordinates)
        best_idx = np.argmax(fitness_scores)
        best_path = population[best_idx]
        best_distance = 1 / fitness_scores[best_idx]
        
        # Display generation info
        print(f"Generation {generation + 1} - Best Distance: {best_distance:.4f}")
        # Display best path in current generation
        print(f"Rute: {best_path}\n")

    # Final best solution
    best_idx = np.argmax(fitness_scores)
    best_solution = population[best_idx]
    end_time = time.time()
    computation_time = end_time - start_time
    return best_solution, best_distance, computation_time

# Assuming city_coordinates are defined as before
# Run the genetic algorithm
best_solution, best_distance, computation_time = genetic_algorithm_tsp_with_gpm(city_coordinates)

print("-----------------------------------------------------------------------------------------------------------------------------")
print(f"Best Distance: {best_distance}")
print(f"Best Solution: {best_solution}")
print(f"Computation Time: {computation_time} seconds")

Generation 1 - Best Distance: 42853.3262
Rute: [23, 34, 16, 8, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 14, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4, 41, 3, 22]

Generation 2 - Best Distance: 42853.3262
Rute: [23, 34, 16, 8, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 14, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4, 41, 3, 22]

Generation 3 - Best Distance: 42853.3262
Rute: [23, 34, 16, 8, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 14, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4, 41, 3, 22]

Generation 4 - Best Distance: 42853.3262
Rute: [23, 34, 16, 8, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 14, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4, 41, 3, 22]

Generation 5 - Best Distance: 42853.3262
Rute: [23, 34, 16, 8, 20, 40, 1

Generation 38 - Best Distance: 42055.8759
Rute: [12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4, 41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15]

Generation 39 - Best Distance: 42055.8759
Rute: [41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4]

Generation 40 - Best Distance: 42055.8759
Rute: [41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4]

Generation 41 - Best Distance: 42055.8759
Rute: [41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4]

Generation 42 - Best Distance: 42055.8759
Rute: [41, 3, 23, 14, 34, 

Generation 79 - Best Distance: 42055.8759
Rute: [41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4]

Generation 80 - Best Distance: 42055.8759
Rute: [41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 26, 4]

Generation 81 - Best Distance: 41866.2555
Rute: [43, 17, 46, 33, 15, 12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 4, 26, 41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27]

Generation 82 - Best Distance: 41866.2555
Rute: [12, 11, 47, 21, 13, 25, 39, 32, 48, 5, 29, 2, 42, 10, 24, 45, 35, 4, 26, 41, 3, 23, 14, 34, 16, 8, 22, 20, 40, 1, 9, 38, 31, 44, 18, 7, 28, 36, 30, 6, 37, 19, 27, 43, 17, 46, 33, 15]

Generation 83 - Best Distance: 41866.2555
Rute: [12, 11, 47, 21, 13,